<a href="https://colab.research.google.com/github/fedelazarte/mi_repositorio/blob/master/Bot_de_Similaridad_de_Jugadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Configuración de la API
API_KEY = "d20aad4a11504e63b933271c0576028b"
BASE_URL = "https://api.football-data.org/v4"

headers = {"X-Auth-Token": API_KEY}

In [ ]:
# Obtener todas las competiciones
def get_competitions():
    url = f"{BASE_URL}/competitions"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()["competitions"]
    else:
        print(f"Error {response.status_code}: {response.json()}")
        return []

In [ ]:
# Obtener equipos de una competición
def get_teams(competition_id):
    url = f"{BASE_URL}/competitions/{competition_id}/teams"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()["teams"]
    else:
        print(f"Error {response.status_code}: {response.json()}")
        return []

In [ ]:
# Obtener plantilla de un equipo e incluir el nombre del equipo en cada jugador
def get_team_players(team_id):
    url = f"{BASE_URL}/teams/{team_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        team_name = data.get("name", "Unknown Team")  # Obtener el nombre del equipo
        players = data.get("squad", [])

        # Agregar el nombre del equipo a cada jugador
        for player in players:
            player["teamName"] = team_name

        return players
    else:
        print(f"Error {response.status_code}: {response.json()}")
        return []


In [ ]:
# Generar DataFrame de equipos
def generate_teams_df(competitions):
    teams = []
    for competition in competitions:
        competition_teams = get_teams(competition["id"])
        for team in competition_teams:
            teams.append({
                "TeamName": team["name"],
                "TeamID": team["id"],
                "CompetitionID": competition["id"],
                "CompetitionName": competition["name"]
            })
    return pd.DataFrame(teams)

In [ ]:
# Generar DataFrame de jugadores
def generate_players_df(teams):
    players = []
    for _, team in teams.iterrows():
        team_id = team["TeamID"]
        team_players = get_team_players(team_id)
        for player in team_players:
            players.append({
                "PlayerName": player["name"],
                "PlayerID": player["id"],
                "TeamID": team_id,
                "TeamName": player["teamName"],  # Incluye el nombre del equipo directamente del jugador
                "Position": player.get("position", "Unknown"),
                "Nationality": player["nationality"]
            })
    return pd.DataFrame(players)


In [ ]:
# Obtener competiciones activas
competitions = get_competitions()

if competitions:
    # Filtrar solo competiciones relevantes (por ejemplo, ligas nacionales)
    competitions = [c for c in competitions if c["type"] == "LEAGUE"]

    # Generar DataFrame de equipos
    teams_df = generate_teams_df(competitions)
    print("Equipos obtenidos:")
    print(teams_df.head())

    # Guardar equipos en CSV
    teams_df.to_csv("teams.csv", index=False)

    # Generar DataFrame de jugadores
    players_df = generate_players_df(teams_df)
    print("Jugadores obtenidos:")
    print(players_df.head())

    # Guardar jugadores en CSV
    players_df.to_csv("players.csv", index=False)
else:
    print("No se pudieron obtener competiciones.")

Equipos obtenidos:
        TeamName  TeamID  CompetitionID                CompetitionName
0  Fluminense FC    1765           2013  Campeonato Brasileiro Série A
1     CA Mineiro    1766           2013  Campeonato Brasileiro Série A
2    Grêmio FBPA    1767           2013  Campeonato Brasileiro Série A
3  CA Paranaense    1768           2013  Campeonato Brasileiro Série A
4   SE Palmeiras    1769           2013  Campeonato Brasileiro Série A
Error 429: {'message': 'You reached your request limit. Wait 12 seconds.', 'errorCode': 429}
Error 429: {'message': 'You reached your request limit. Wait 12 seconds.', 'errorCode': 429}
Error 429: {'message': 'You reached your request limit. Wait 11 seconds.', 'errorCode': 429}
Error 429: {'message': 'You reached your request limit. Wait 11 seconds.', 'errorCode': 429}
Error 429: {'message': 'You reached your request limit. Wait 10 seconds.', 'errorCode': 429}
Error 429: {'message': 'You reached your request limit. Wait 10 seconds.', 'errorCode': 42

In [ ]:
# Procesar datos de los jugadores
def process_players(players, team_name):
    player_data = []
    for player in players:
        player_data.append({
            "Name": player["name"],
            "Position": player.get("position", "Unknown"),
            "DateOfBirth": player["dateOfBirth"],
            "Nationality": player["nationality"],
            "Team": team_name  # Añadir el nombre del equipo directamente
        })
    return pd.DataFrame(player_data)

In [ ]:
# Obtener y procesar datos iniciales
TEAM_ID = 1765  # Ejemplo: Fluminense
players = get_team_players(TEAM_ID)


In [ ]:
# Función para obtener posiciones similares
def get_similar_positions(position):
    similar_positions = {
        "Forward": ["Forward", "Winger"],
        "Midfielder": ["Midfielder", "Winger", "Defensive Midfielder"],
        "Defender": ["Defender", "Wing Back", "Defensive Midfielder"],
        "Goalkeeper": ["Goalkeeper"]
    }
    return similar_positions.get(position, [position])

In [ ]:
# Función para recomendar jugadores similares
def recommend_players(player_identifier, data, top_n=5):
    # Buscar jugador por nombre o ID
    if isinstance(player_identifier, str):  # Nombre
        player_data = data[data["Name"] == player_identifier]
    else:  # ID
        player_data = data[data.index == player_identifier]

    if player_data.empty:
        return f"El jugador con identificador '{player_identifier}' no existe en la base de datos."

    # Obtener la posición del jugador objetivo
    target_position = player_data.iloc[0]["Position"]
    similar_positions = get_similar_positions(target_position)

    # Filtrar jugadores con posiciones similares
    position_filtered_data = data[data["Position"].isin(similar_positions)]

    if position_filtered_data.empty or len(position_filtered_data) == 1:
        return f"No hay suficientes jugadores en posiciones similares para generar recomendaciones."

    # Calcular similitudes solo con jugadores de posiciones similares
    similarities = cosine_similarity(player_data[attributes], position_filtered_data[attributes])
    position_filtered_data = position_filtered_data.copy()
    position_filtered_data["Similarity"] = similarities[0]

    # Ordenar por similitud y devolver los más similares
    recommendations = position_filtered_data.sort_values(by="Similarity", ascending=False).head(top_n + 1)
    return recommendations[["Name", "Position", "Team", "Nationality", "Similarity"]].iloc[1:]


In [ ]:
# Función para mostrar un menú desplegable con los jugadores del equipo
def display_player_selection(team_id, data):
    # Filtrar jugadores del equipo
    team_players = data[data["TeamID"] == team_id]

    if team_players.empty:
        print("No se encontraron jugadores para el equipo seleccionado.")
        return None

    # Crear un menú desplegable con los nombres de los jugadores
    player_dropdown = widgets.Dropdown(
        options=[(row["Name"], row["PlayerID"]) for _, row in team_players.iterrows()],
        description="Jugador:",
        style={"description_width": "initial"},
        layout={"width": "400px"}
    )

    # Función para mostrar recomendaciones basadas en el jugador seleccionado
    def on_player_selected(change):
        player_id = change["new"]
        if player_id:
            recommendations = recommend_players(player_id, data)
            print(f"\nRecomendaciones para el jugador seleccionado:")
            print(recommendations)

    # Conectar el evento de selección del jugador
    player_dropdown.observe(on_player_selected, names="value")
    display(player_dropdown)

In [ ]:
# Generar y mostrar el menú desplegable
display_player_selection(TEAM_ID, df_players)

KeyError: 'TeamID'